#### Data Loading and Exploration

##### Import required libraries

In [83]:
###basic python librarries 
import os      ###for folder path operation 
import warnings           #to ignore warning message
warnings.filterwarnings('ignore')

In [84]:
## Data Manipulation and analysis
import pandas as pd
import numpy as np

In [85]:
#### Exploratory Data Analysis
#statistical summary 
from scipy import stats 
from scipy.stats import skew,kurtosis

In [86]:
##display setting
pd.set_option('display.max_columns',None)
pd.set_option('display.width',1000)

In [87]:
##library for connection
import sqlite3
from sqlalchemy import create_engine


In [88]:
print("Finally every required libraries are imported")

Finally every required libraries are imported


#### Load all csv files

In [89]:
Folder_path=r"C:\Users\Asus\OneDrive\Desktop\Flipkart electronic product2025(sqllite+etl+power bi)\data"

In [90]:
earphones=pd.read_csv(os.path.join(Folder_path,"flipkart_earphones.csv"))
laptops=pd.read_csv(os.path.join(Folder_path,"flipkart_laptops.csv"))
mobiles=pd.read_csv(os.path.join(Folder_path,"flipkart_mobile_data.csv"))             

#### Basic Explorartory Data Analyst( Check structure of each dataset)

In [91]:
for name ,df in {
    "Earphones":earphones,
    "Laptops":laptops,
    "Mobiles":mobiles
}.items():
    print(f"---{name}---")
    print(df.shape)
    print(df.isnull().sum())
    print(df.columns)
    print()

---Earphones---
(960, 10)
Title               0
Product URL         0
Image URL           0
Rating            145
Rating Count      145
Price              84
Original Price     87
Discount           87
Offer              84
Timestamp           0
dtype: int64
Index(['Title', 'Product URL', 'Image URL', 'Rating', 'Rating Count', 'Price', 'Original Price', 'Discount', 'Offer', 'Timestamp'], dtype='object')

---Laptops---
(540, 12)
Title                  0
Product Link           0
Image URL              0
Rating               105
Ratings & Reviews    105
Price                  0
Original Price         5
Discount               6
Key Features           0
Exchange Offer       394
Bank Offer            29
Timestamp              0
dtype: int64
Index(['Title', 'Product Link', 'Image URL', 'Rating', 'Ratings & Reviews', 'Price', 'Original Price', 'Discount', 'Key Features', 'Exchange Offer', 'Bank Offer', 'Timestamp'], dtype='object')

---Mobiles---
(960, 12)
Title                  0
Product Link

In [92]:
### Now start to handle missing values ,outlier with first table(earphones)

In [93]:
### So now after filling missing value i got an error bcz in the numeric col there r commas and symbols are present....
#so i have decided to removed unwanted chaacters like ₹, %,nd commas from numeric cols to make them
#consistent nd ready for mathematical annalysis nd visualll


In [94]:
## remove unneccessary characters nd convert it into numeric
earphones['Rating Count']=earphones['Rating Count'].astype(str).str.replace(r'[^\d]','',regex=True)
earphones['Rating Count']=pd.to_numeric(earphones['Rating Count'],errors='coerce')

earphones['Price'] = earphones['Price'].astype(str).str.replace(r'[^\d]', '', regex=True)
earphones['Price'] = pd.to_numeric(earphones['Price'], errors='coerce')

earphones['Original Price'] = earphones['Original Price'].astype(str).str.replace(r'[^\d]', '', regex=True)
earphones['Original Price'] = pd.to_numeric(earphones['Original Price'], errors='coerce')

earphones['Discount'] = earphones['Discount'].astype(str).str.replace(r'[^0-9]', '', regex=True)
earphones['Discount'] = pd.to_numeric(earphones['Discount'], errors='coerce')


In [95]:
# Numeric cols-fill the numeric cols with median 


In [96]:
earphones['Rating'].fillna(earphones['Rating'].median(),inplace=True)
earphones['Rating Count'].fillna(earphones['Rating Count'].median(),inplace=True)
earphones['Price'].fillna(earphones['Price'].median(),inplace=True)
earphones['Original Price'].fillna(earphones['Original Price'].median(),inplace=True)
earphones['Discount'].fillna(earphones['Discount'].median(),inplace=True)

In [97]:
# Categorical cols-----fill with mode or default
earphones['Offer'].fillna('No offer',inplace=True)

In [98]:
earphones.isnull().sum()


Title             0
Product URL       0
Image URL         0
Rating            0
Rating Count      0
Price             0
Original Price    0
Discount          0
Offer             0
Timestamp         0
dtype: int64

In [99]:
## Now Detect outliers 
numeric_cols=earphones.select_dtypes(include=['float64','int64']).columns
for col in numeric_cols:
    Q1=earphones[col].quantile(0.25)
    Q3=earphones[col].quantile(0.75)
    IQR=Q3-Q1
    lower_limit=Q1 -1.5*IQR
    upper_limit=Q3 +1.5*IQR
    outliers=earphones[(earphones[col]<lower_limit) | (earphones[col]> upper_limit)]
    print(f"Number of outliers in {col}:{outliers.shape[0]}")



Number of outliers in Rating:101
Number of outliers in Rating Count:119
Number of outliers in Price:50
Number of outliers in Original Price:33
Number of outliers in Discount:58


In [100]:
# i think so i should capp them via iqr method

In [101]:
for col in numeric_cols:
    Q1=earphones[col].quantile(0.25)
    Q3=earphones[col].quantile(0.75)
    IQR=Q3-Q1
    lower=Q1 - 1.5*IQR
    upper=Q3 - 1.5*IQR
    earphones[col]=np.where(earphones[col]<lower,lower,earphones[col])
    earphones[col]=np.where(earphones[col]>upper,upper,earphones[col])

In [102]:
for col in numeric_cols:
    Q1=earphones[col].quantile(0.25)
    Q3=earphones[col].quantile(0.75)
    IQR=Q3-Q1
    lower_limit=Q1 -1.5*IQR
    upper_limit=Q3 +1.5*IQR
    outliers=earphones[(earphones[col]<lower_limit) | (earphones[col]> upper_limit)]
    print(f"Number of outliers in {col}:{outliers.shape[0]}")

Number of outliers in Rating:119
Number of outliers in Rating Count:0
Number of outliers in Price:0
Number of outliers in Original Price:52
Number of outliers in Discount:151


In [103]:
#I handled the outliers using the IQR-based capping technique 
# which means the dataset is now completely clean and ready for further analysis

In [104]:
### handle duplicate if exists
earphones.duplicated().sum()

0

In [105]:
## Move ahead another one's (laptops)
import re
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              540 non-null    object 
 1   Product Link       540 non-null    object 
 2   Image URL          540 non-null    object 
 3   Rating             435 non-null    float64
 4   Ratings & Reviews  435 non-null    object 
 5   Price              540 non-null    object 
 6   Original Price     535 non-null    object 
 7   Discount           534 non-null    object 
 8   Key Features       540 non-null    object 
 9   Exchange Offer     146 non-null    object 
 10  Bank Offer         511 non-null    object 
 11  Timestamp          540 non-null    object 
dtypes: float64(1), object(11)
memory usage: 50.8+ KB


In [106]:
# first of all we should split the col -(ratings and reviews) 
def split_rating_review(value):
    if pd.isna(value):
        return pd.Series([np.nan,np.nan])
    match =re.match(r"(\d+)\s*Ratings\s*&\s*(\d+)\s*Reviews",str(value))
    if match:
        return pd.Series([int(match.group(1)),int(match.group(2))])
    else:
        return pd.Series([np.nan,np.nan])
laptops[['Rating_Count','Review_count']]=laptops['Ratings & Reviews'].apply(split_rating_review)

In [107]:
## Clean Price ,original price,Discount
def clean_price(value):
    if pd.isna(value):
        return np.nan
    return float(str(value).replace('₹','').replace(',','').strip())

def clean_discount(value):
    if pd.isna(value):
        return np.nan
    return float(str(value).replace('%','').replace('off','').strip())

laptops['Price'] = laptops['Price'].apply(clean_price)
laptops['Original Price'] = laptops['Original Price'].apply(clean_price)
laptops['Discount'] = laptops['Discount'].apply(clean_discount)

In [108]:
# Convert Timestamp to Datetime
laptops['Timestamps']=pd.to_datetime(laptops['Timestamp'],errors="coerce")

In [109]:
## Now it's time to handle missing value 
laptops['Rating'].fillna(laptops['Rating'].median(), inplace=True)
laptops['Rating_Count'].fillna(laptops['Rating_Count'].median(), inplace=True)
laptops['Review_count'].fillna(laptops['Review_count'].median(), inplace=True)
laptops['Price'].fillna(laptops['Price'].median(), inplace=True)
laptops['Original Price'].fillna(laptops['Original Price'].median(), inplace=True)
laptops['Discount'].fillna(laptops['Discount'].median(), inplace=True)

In [110]:
print(laptops.columns)

Index(['Title', 'Product Link', 'Image URL', 'Rating', 'Ratings & Reviews', 'Price', 'Original Price', 'Discount', 'Key Features', 'Exchange Offer', 'Bank Offer', 'Timestamp', 'Rating_Count', 'Review_count', 'Timestamps'], dtype='object')


In [111]:
# Categorical missing value handle 
laptops['Exchange Offer'].fillna('No offer',inplace=True)
laptops['Bank Offer'].fillna('No offer',inplace=True)

In [112]:
### outliers detection 
numeric_colls=['Rating','Rating_Count','Review_count','Price','Original Price','Discount']
for col in numeric_colls:
    Q1=laptops[col].quantile(0.25)
    Q3=laptops[col].quantile(0.75)
    IQR = Q3-Q1
    lower = Q1 - 1.5*IQR
    upper= Q3 + 1.5*IQR
    outliers=laptops[(laptops[col]<lower) | (laptops[col]>upper)]
    print(f"{col}:{len(outliers)} outliers")

Rating:23 outliers
Rating_Count:60 outliers
Review_count:53 outliers
Price:9 outliers
Original Price:32 outliers
Discount:0 outliers


In [113]:
### Now i'm going to apply capping/winsorization
for col in numeric_colls:
    Q1=laptops[col].quantile(0.25)
    Q3=laptops[col].quantile(0.75)
    IQR = Q3-Q1
    lower = Q1 - 1.5*IQR
    upper= Q3 + 1.5*IQR
    laptops[col]=np.where(laptops[col]<lower,lower,laptops[col])
    laptops[col]=np.where(laptops[col]>upper,upper,laptops[col])


In [114]:
for col in numeric_colls:
    Q1=laptops[col].quantile(0.25)
    Q3=laptops[col].quantile(0.75)
    IQR = Q3-Q1
    lower = Q1 - 1.5*IQR
    upper= Q3 + 1.5*IQR
    outliers=laptops[(laptops[col]<lower) | (laptops[col]>upper)]
    print(f"{col}:{len(outliers)} outliers")

Rating:0 outliers
Rating_Count:0 outliers
Review_count:0 outliers
Price:0 outliers
Original Price:0 outliers
Discount:0 outliers


In [115]:
### initially some outliers were detected in columns ..
## after applyuing the IQR capping method,alll cols now show 0 outliers 
#which indicates that the data distribution is normalized and read for queriz"

In [116]:
##handle duplicate if exists 
laptops.duplicated().sum()

0

In [117]:
## next table(mobiles)
mobiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              960 non-null    object 
 1   Product Link       960 non-null    object 
 2   Image URL          960 non-null    object 
 3   Rating             960 non-null    float64
 4   Ratings & Reviews  960 non-null    object 
 5   Price              960 non-null    object 
 6   Original Price     938 non-null    object 
 7   Discount           936 non-null    object 
 8   Key Features       960 non-null    object 
 9   Exchange Offer     530 non-null    object 
 10  Bank Offer         915 non-null    object 
 11  Timestamp          960 non-null    object 
dtypes: float64(1), object(11)
memory usage: 90.1+ KB


In [118]:
# first of all we should split the col -(ratings and reviews) 
def split_rating_review(value):
    if pd.isna(value):
        return pd.Series([np.nan,np.nan])
    match =re.match(r"(\d+)\s*Ratings\s*&\s*(\d+)\s*Reviews",str(value))
    if match:
        return pd.Series([int(match.group(1)),int(match.group(2))])
    else:
        return pd.Series([np.nan,np.nan])
mobiles[['Rating_Count','Review_count']]=mobiles['Ratings & Reviews'].apply(split_rating_review)

In [119]:
# Convert Timestamp to Datetime
mobiles['Timestamps']=pd.to_datetime(mobiles['Timestamp'],errors="coerce")

In [120]:
## Clean Price ,original price,Discount
def clean_price(value):
    if pd.isna(value):
        return np.nan
    return float(str(value).replace('₹','').replace(',','').strip())

def clean_discount(value):
    if pd.isna(value):
        return np.nan
    return float(str(value).replace('%','').replace('off','').strip())

mobiles['Price'] = mobiles['Price'].apply(clean_price)
mobiles['Original Price'] = mobiles['Original Price'].apply(clean_price)
mobiles['Discount'] = mobiles['Discount'].apply(clean_discount)

In [121]:
#handle missing value 
mobiles['Original Price'].fillna(mobiles['Original Price'].median(), inplace=True)
mobiles['Discount'].fillna(mobiles['Discount'].median(), inplace=True)

In [122]:
# Categorical missing value handle 
mobiles['Exchange Offer'].fillna('No offer',inplace=True)

In [123]:
## handle duplicate value
mobiles.duplicated().sum()

0

In [124]:
### Now all tables are capped nd handle turn to next step

#### Export Cleaned data in the csv file

In [125]:
earphones.to_csv(r"C:\Users\Asus\OneDrive\Desktop\Flipkart electronic product2025(sqllite+etl+power bi)\data\Cleanes_file\earphones_cleaned.csv",index=False)
laptops.to_csv(r"C:\Users\Asus\OneDrive\Desktop\Flipkart electronic product2025(sqllite+etl+power bi)\data\Cleanes_file\laptops_cleaned.csv",index=False)
mobiles.to_csv(r"C:\Users\Asus\OneDrive\Desktop\Flipkart electronic product2025(sqllite+etl+power bi)\data\Cleanes_file\mobiles_cleaned.csv",index=False)

In [126]:
## data import in ssllite
import sqlite3
from sqlalchemy import create_engine

In [135]:
###SQLite data base connect 
db_path=r"C:\Users\Asus\OneDrive\Desktop\Flipkart electronic product2025(sqllite+etl+power bi)\sqllite\flipkart_electronic_products.db"
conn=sqlite3.connect(db_path)
cursor=conn.cursor()

In [136]:
###earphn 
df=pd.read_csv(r"C:\Users\Asus\OneDrive\Desktop\Flipkart electronic product2025(sqllite+etl+power bi)\data\Cleanes_file\earphones_cleaned.csv")
df2=pd.read_csv(r"C:\Users\Asus\OneDrive\Desktop\Flipkart electronic product2025(sqllite+etl+power bi)\data\Cleanes_file\laptops_cleaned.csv")
df3=pd.read_csv(r"C:\Users\Asus\OneDrive\Desktop\Flipkart electronic product2025(sqllite+etl+power bi)\data\Cleanes_file\mobiles_cleaned.csv")

In [149]:
##Rename cols to match db table
df.rename(columns={
    'Title':'Title',
    'Product URL':'Product_URL',
    'Image URL':'Image_URL',
    'Rating':'Rating',
    'Rating Count':'Rating_Count',
    'Price':'Price',
    'Original Price':'Original_Price',
    'Discount':'Discount',
    'Offer':'Offer',
    'Timestamp':'Timestamp'
},inplace=True)
df2.rename(columns={
    'Title':'Title',
    'Product Link':'Product_Link',
    'Image URL':'Image_URL',
    'Rating':'Rating',
    'Ratings & Reviews':'Ratings_Reviews',
    'Price':'Price',
    'Original Price':'Original_Price',
    'Discount':'Discount',
    'Key Features':'Key_Features',
    'Exchange Offer':'Exchange_Offer',
    'Bank Offer':'Bank_Offer',
    'Timestamp':'Timestamp',
    'Rating_Count':'Rating_Count',
    'Review_count':'Review_count',
    'Timestamps':'Timestamps'
},inplace=True)
df3.rename(columns={
    'Title':'Title',
    'Product Link':'Product_Link' ,
    'Image URL':'Image_URL',
    'Rating':'Rating',
    'Ratings & Reviews':'Ratings_Reviews',
    'Price':'Price',
    'Original Price':'Original_Price',
    'Discount':'Discount',
    'Key Features':'Key_Features',
    'Exchange Offer':'Exchange_Offer',
    'Bank Offer':'Bank_Offer',
    'Timestamp':'Timestamp',
    'Rating_Count':'Rating_Count',
    'Review_count':'Review_count',
    'Timestamps':'Timestamps'
},inplace= True)     

In [144]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960 entries, 0 to 959
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              960 non-null    object 
 1   Product Link       960 non-null    object 
 2   Image URL          960 non-null    object 
 3   Rating             960 non-null    float64
 4   Ratings & Reviews  960 non-null    object 
 5   Price              960 non-null    float64
 6   Original Price     960 non-null    float64
 7   Discount           960 non-null    float64
 8   Key Features       960 non-null    object 
 9   Exchange Offer     960 non-null    object 
 10  Bank Offer         915 non-null    object 
 11  Timestamp          960 non-null    object 
 12  Rating_Count       273 non-null    float64
 13  Review_count       273 non-null    float64
 14  Timestamps         960 non-null    object 
dtypes: float64(6), object(9)
memory usage: 112.6+ KB


In [150]:
df.to_sql('earphones',conn,if_exists='append',index=False)
df2.to_sql('laptops',conn,if_exists='append',index=False)
df3.to_sql('mobiles',conn,if_exists='append',index=False)
conn.commit()
conn.close()
print("All three tables loaded ")

All three tables loaded 
